In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-quiz1-p2/train.csv
/kaggle/input/2021-ml-quiz1-p2/test.csv
/kaggle/input/2021-ml-quiz1-p2/sample_submit.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/2021-ml-quiz1-p2/train.csv')
test_data = pd.read_csv('/kaggle/input/2021-ml-quiz1-p2/test.csv')
submit = pd.read_csv('/kaggle/input/2021-ml-quiz1-p2/sample_submit.csv')

print(train_data.head())
print(test_data.head())

   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        4      Auris  2015             1    47541         4  145  46.3   
1   1        0         X2  2018             3     5000         0  145  50.4   
2   2        2      Focus  2017             3    47018         4  145  51.4   
3   3        2      Focus  2020             1     1550         4  145  49.6   
4   4        0   3 Series  2017             3    23505         0  145  64.2   

   engineSize  price  
0         1.6   8095  
1         2.0  21726  
2         1.0  10490  
3         1.0  17490  
4         2.0  18995  
   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2        Ka+  2017             1    10150         4  145  57.7   
1   1        2       Kuga  2013             1    50216         0  145  53.3   
2   2        3         Q5  2016             1    34189         0  200  47.9   
3   3        1        I10  2020             1     1900         4  145  

In [3]:
x_data = train_data.drop(['ID', 'price'], axis=1)
test_data= test_data.drop('ID', axis=1)
y_data = train_data['price']
print(x_data.head())
print(test_data.head())

   company      model  year  transmission  mileage  fueltype  tax   mpg  \
0        4      Auris  2015             1    47541         4  145  46.3   
1        0         X2  2018             3     5000         0  145  50.4   
2        2      Focus  2017             3    47018         4  145  51.4   
3        2      Focus  2020             1     1550         4  145  49.6   
4        0   3 Series  2017             3    23505         0  145  64.2   

   engineSize  
0         1.6  
1         2.0  
2         1.0  
3         1.0  
4         2.0  
   company      model  year  transmission  mileage  fueltype  tax   mpg  \
0        2        Ka+  2017             1    10150         4  145  57.7   
1        2       Kuga  2013             1    50216         0  145  53.3   
2        3         Q5  2016             1    34189         0  200  47.9   
3        1        I10  2020             1     1900         4  145  56.5   
4        0   3 Series  2014             0    79809         0   20  68.9   

  

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_data_le = le.fit_transform(x_data['model'].values)
test_data_le = le.transform(test_data['model'].values)

x_data['model'] = x_data_le
test_data['model'] = test_data_le

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_data['mileage'].values.reshape(-1, 1))

x_data_mileage = scaler.transform(x_data['mileage'].values.reshape(-1, 1))
test_data_mileage = scaler.transform(test_data['mileage'].values.reshape(-1, 1))

x_data['mileage'] = x_data_mileage
test_data['mileage'] = test_data_mileage

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size = 0.3, random_state=123)
print(x_train.shape)
print(x_val.shape)

(23919, 9)
(10251, 9)


In [7]:
from sklearn.neighbors import KNeighborsRegressor
reg = KNeighborsRegressor(n_neighbors = 5, weights = 'distance' , p=1) # neightbors = 5

reg.fit(x_train, y_train)
val_pred = reg.predict(x_val)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_val, val_pred))


1306.8714870575827


In [8]:
# train
reg.fit(x_data, y_data)
pred = reg.predict(test_data)

print(pred)

[ 8323.44663331  9212.25270542 20022.20081524 ... 14254.3324794
 29781.75053937 12427.02345439]


In [9]:
submit['price'] = pred
submit

,ID,price
0,0,8323.446633
1,1,9212.252705
2,2,20022.200815
3,3,12141.699542
4,4,9410.894365
...,...,...
16826,16826,13055.437173
16827,16827,12231.341051
16828,16828,14254.332479
16829,16829,29781.750539


In [10]:
submit.to_csv('submit.csv', header = True, index = False)